In [ ]:
import pandas as pd
import os

pd.set_option('display.max_rows', 10)
pd.set_option('display.max_columns', 10)

## Generation timeseries 
- Imports timeseries for different timeframes
- Converts str into numeric dataframe

In [ ]:
cwd = os.getcwd()
generation_2018_path = os.path.join(cwd, "generation-20180101-20180401.csv")
generation_2019_path = os.path.join(cwd, "generation-20190101-20190401.csv")
generation_2020_path = os.path.join(cwd, "generation-20200101-20200401.csv")
generation_2021_path = os.path.join(cwd, "generation-20210101-20210401.csv")
generation_2022_path = os.path.join(cwd, "generation-20220101-20220401.csv")
generation_t1_path = os.path.join(cwd, "generation_gas_germany_2019.csv")

generation_2018 = pd.read_csv(generation_2018_path, index_col=0)
generation_2019 = pd.read_csv(generation_2019_path, index_col=0)
generation_2020 = pd.read_csv(generation_2020_path, index_col=0)
generation_2021 = pd.read_csv(generation_2021_path, index_col=0)
generation_2022 = pd.read_csv(generation_2022_path, index_col=0)
generation_t1 = pd.read_csv(generation_t1_path, index_col=0)

In [ ]:
generation_2018 = generation_2018.iloc[2:,:]
generation_2018 = generation_2018.apply(pd.to_numeric)

generation_2019 = generation_2019.iloc[2:,:]
generation_2019 = generation_2019.apply(pd.to_numeric)

generation_2020 = generation_2020.iloc[2:,:]
generation_2020 = generation_2020.apply(pd.to_numeric)

generation_2021 = generation_2021.iloc[2:,:]
generation_2021 = generation_2021.apply(pd.to_numeric)

generation_2022 = generation_2022.iloc[2:,:]
generation_2022 = generation_2022.apply(pd.to_numeric)

In [ ]:
df2018 = pd.DataFrame(generation_2018.sum(axis=1).values)
df2018["year"] = pd.DatetimeIndex(generation_2018.index.values).year
xlabel = pd.DataFrame({'snapshot':pd.date_range("2019-01-01", periods=len(generation_2022), freq="H")})
df2018["time"] = xlabel.snapshot.dt.strftime('%d-%B %H:00')

df2019 = pd.DataFrame(generation_2019.sum(axis=1).values)
df2019["year"] = pd.DatetimeIndex(generation_2019.index.values).year
xlabel = pd.DataFrame({'snapshot':pd.date_range("2019-01-01", periods=len(generation_2022), freq="H")})
df2019["time"] = xlabel.snapshot.dt.strftime('%d-%B %H:00')

df2020 = pd.DataFrame(generation_2020.sum(axis=1).values)
df2020["year"] = pd.DatetimeIndex(generation_2020.index.values).year
xlabel = pd.DataFrame({'snapshot':pd.date_range("2019-01-01", periods=len(generation_2022), freq="H")})
df2020["time"] = xlabel.snapshot.dt.strftime('%d-%B %H:00')

df2021 = pd.DataFrame(generation_2021.sum(axis=1).values)
df2021["year"] = pd.DatetimeIndex(generation_2021.index.values).year
xlabel = pd.DataFrame({'snapshot':pd.date_range("2019-01-01", periods=len(generation_2022), freq="H")})
df2021["time"] = xlabel.snapshot.dt.strftime('%d-%B %H:00')

df2022 = pd.DataFrame(generation_2022.sum(axis=1).values)
df2022["year"] = pd.DatetimeIndex(generation_2022.index.values).year
xlabel = pd.DataFrame({'snapshot':pd.date_range("2019-01-01", periods=len(generation_2022), freq="H")})
df2022["time"] = xlabel.snapshot.dt.strftime('%d-%B %H:00')


frames = [df2018, df2019, df2020, df2021, df2022]
df_total = pd.concat(frames).rename(columns={0:"p"})
df_total = df_total.reset_index().rename(columns={"index":"snapshot"})
df_total["p"] = df_total["p"].rolling(24*7,center=True,min_periods=1).mean()/1000  # 1 week smoothing and GW conversion


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme(style="dark")


# Plot each year's time series in its own facet
g = sns.relplot(
    data=df_total,
    x="time", y="p", col="year", hue="year",
    kind="line", palette="crest", linewidth=4, zorder=5,
    col_wrap=1, height=3, aspect=3, legend=False,
)

# Iterate over each subplot to customize further
for year, ax in g.axes_dict.items():

    if year == 2018:
        ax.set_title("Main power output of EU Combined-Heat and Power plant fleet")
    else:
        ax.set_title("")
    # Add the title as an annotation within the plot
    ax.text(.8, .85, year, transform=ax.transAxes, fontweight="bold")

    # Plot every year's time series in the background
    sns.lineplot(
        data=df_total, x="time", y="p", units="year",
        estimator=None, color=".7", linewidth=1, ax=ax,
    )

# Reduce the frequency of the x axis ticks
ticks = [
df_total[df_total["time"]=="01-January 00:00"].index[0],
df_total[df_total["time"]=="01-February 00:00"].index[0],
df_total[df_total["time"]=="01-March 00:00"].index[0],
df_total[df_total["time"]=="31-March 00:00"].index[0],
]
ax.set_xticks(ticks)

# Tweak the supporting aspects of the plot

g.set_axis_labels("", "Power [GW_el]")
g.tight_layout()
plt.xlim([0, len(generation_2022)])



In [ ]:
[
df_total[df_total["time"]=="01-January 00:00"].index[0],
df_total[df_total["time"]=="01-February 00:00"].index[0],
df_total[df_total["time"]=="01-March 00:00"].index[0],
df_total[df_total["time"]=="31-March 00:00"].index[0]
]

In [ ]:
df_total["time"]

## Generation capacity
- Import powerplants
- Filters Germany
- Filter CHP 

In [ ]:
cwd = os.getcwd()
ppm_generation_path = os.path.join(cwd, "ppm-powerplants.csv")
pypsaeur_generation_path = os.path.join(cwd, "pypsaeur-powerplants.csv")
ppm_powerplants = pd.read_csv(ppm_generation_path, index_col=0)
ppm_powerplants

In [ ]:
ppm_powerplants = ppm_powerplants.loc[ppm_powerplants["Country"]=="Germany", :]
ppm_powerplants = ppm_powerplants.loc[ppm_powerplants["Set"]=="CHP", :]
ppm_powerplants = ppm_powerplants.loc[ppm_powerplants["Fueltype"]=="Natural Gas", :]

In [ ]:
ppm_powerplants
